In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Définition de la classe abstraite

c'est une couche "type" qu'il faut remplir/définir par la suite en fonction de la couche que l'on veut coder  

In [ ]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None

    # calcule de la sortie Y pour une entrée X donnée 
    def forward_propagation(self, X):
        raise NotImplementedError #Donc on ne définit pas encore la fonction, ce sera fait par la suite

    # calcule de dE/dX 
    def backward_propagation(self, dE_dY , alpha):
        raise NotImplementedError #Donc on ne définit pas encore la fonction, ce sera fait par la suite

# Couche FC (Fully Connected)

On se limite à des couche FC (càd des couches où tous les neurones d'entrée sont connectés avec tous ceux de sortie)

In [ ]:
# Remarque pour la synthaxe : dE_dY = dE/dY, idem pour dE_dX et dE_dW 

class FCLayer(Layer):
    # définitions des données 
    def __init__(self, nb_neurones_X, nb_neurones_Y):
        self.weights = np.random.rand(nb_neurones_X, nb_neurones_Y) - 0.5
        self.bias = np.random.rand(1, nb_neurones_Y) - 0.5

    # calcule de la sortie Y pour une entrée X donnée     
    def forward_propagation(self, input_data):
        self.input = input_data 
        #on calcule la sortie avec la formule classique Y = XW + B
        self.output = np.dot(self.input, self.weights) + self.bias 
        return self.output

    # calcule de dE/dX 
    def backward_propagation(self, dE_dY, alpha): 
        #dE/dX et dE/dW servent à ajuster le Poids (weight) et le Biais afin d'augmenter les performances du réseau 
        dE_dX = np.dot(dE_dY, self.weights.T) 
        dE_dW = np.dot(self.input.T, dE_dY)

        # ajustement des paramètres comme annoncé précédement
        self.weights -= alpha * dE_dW
        self.bias -= alpha * dE_dY
        return dE_dX

# Couche activation

qui par la fonction d'activation produit un effet de seuil permettant une modification des poids seulement si certaines valeurs sont dépassées

In [ ]:
class ActivationLayer (Layer) : #On prend en entrée un variable de type de la classe Layer

    #On initialise la couche avec la fonction d'activation, sa dérivée pour le calcul de l'erreur et param sera l'objet contenant les infos
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime


    #On code la passe avant, c'est un simple calcul d'image par la fonction d'activation   
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input) #On calcul la sortie en donnant l'entrée à  la fonction d'activ 
        return self.output

    def backward_propagation(self, output_error, learning_rate):
        #Le calcul de dE/dY nous permet de définir la fonction pour la régression
        return self.activation_prime(self.input) * output_error

### Tangente hyperbolique

#De manière pas très originale, on peut reprendre la même fonction d'activation que lui, dont la forme modélise bien l'effet de seuil recherché pour une telle fonction.
def tanh(x): 
    return np.tanh(x)

def tanh_prime(x):
    return 1-np.tanh(x)**2

### Sigmoïd

import math

def sig(x):
    return 1 / (1 + math.exp(-x))

def sig_prime(x) : 
    return math.exp(x)/(1+math.exp(x))**2

### ReLU

In [ ]:
def ReLU(x):
    return x * (x > 0)

def dReLU(x):
    return 1. * (x > 0)

In [ ]:
#Avec un fonction de perte classique (MSE)
#Rq : on la modifiera pour optimiser le programme, comme la fonction d'activation

def mse(y_true, y_pred):
    return np.mean(np.power(y_true-y_pred, 2));

def mse_prime(y_true, y_pred):
    return 2*(y_pred-y_true)/y_true.size

# Classe Network

c'est la construction du réseau de neurones en utilisant 

In [ ]:
class Network:
    def __init__(self):
        self.layers = []              # On créé dans self une liste qui contiendra toutes les couches de neurones
        self.loss = None              # On définira la fonction de perte et sa dérivée ultérieurment
        self.loss_prime = None

    def add(self, layer):             # On définit la fonction d'addition d'une couche au réseau de neurones
        self.layers.append(layer)

    def use(self, loss, loss_prime):  # On créé la fonction permettant de définir la fonction de perte à utiliser pour le réseau
        self.loss = loss              # On choisit l'argument loss renseigné dans la fonction comme fonction de perte, de même pour sa dérivée
        self.loss_prime = loss_prime

    def predict(self, input_data):    # Création de la fonction de prédiction d'une sortie result pour une entrée input_data donnée
        samples = len(input_data)     # La taille de la sortie doit faire la taille de l'entrée
        result = []                   # Création de la liste dans laquelle sera stockée la sortie

        for i in range(samples):      # Boucle de la prédiction des valeurs en sortie
            output = input_data[i]
            for layer in self.layers: # On repéte l'itération pour toutes les couches afin de propager l'entrée
                output = layer.forward_propagation(output) # On utilise la passe avant pour obtenir la sortie
            result.append(output)     # On obtient la sortie en dernière couche

        return result

    def fit(self, x_train, y_train, epochs, learning_rate) : #La fonction fit va servir à entraîner le réseau de neuronnes
#On donne le nombre d'aller-retour qu'on veut effectuer avec la variable epoch

        samples = len(x_train) #On prend les dimensions des paramètre en input

        for i in range(epochs):
            err = 0 #On va devoir calculer l'erreur, qu'on initialise à 0

            for j in range(samples):

                #1ère étape : passe avant dans le réseau :
                output = x_train[j] #On va calculer la sortie pour chaque x_j de l'entrée
                for layer in self.layers:#En bouclant sur les couches, on parcourt le réseau en profondeur
                    output = layer.forward_propagation(output) #Pas besoin de redonner self qui est global dans la structure de class
                err += self.loss(y_train[j], output)
                #2e étape : passe arrière dans le réseau :
                error = self.loss_prime(y_train[j], output) #Fondamentalement pour la descente de gradient, il faut l'erreur, que l'on calcule :
                for layer in reversed(self.layers): #On parcourt depuis la fin donc on doit retourner la liste des couches
                    error = layer.backward_propagation(error,learning_rate) #On utilise les fontions héritées de la structure de couche pour calculer l'erreur

            # erreur moyenne sur tous les échantillons étudiés
            err /= samples
            print('epoch %d/%d   error=%f' % (i+1, epochs, err))

# XOR


# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
net = Network()
net.add(FCLayer(2, 3))
net.add(ActivationLayer(ReLU, dReLU))
net.add(FCLayer(3, 1))
net.add(ActivationLayer(ReLU, dReLU))

# train
net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)

# test
out = net.predict(x_train)
print(out)

# Chargement du dataset des ramens

In [ ]:
ram = pd.read_csv('../input/ramen-ratings/ramen-ratings.csv')

In [ ]:
ram = ram.drop(['Variety'], axis = 1)
ram = ram.drop(['Review #'], axis = 1)
ram = ram.drop(['Brand'], axis = 1)
ram = ram.drop(['Top Ten'], axis = 1)

#On remplace Cup, Pack, Tray, Bowl par 1, 2, 3, 4
#On drop les Unrated
#On remplace Japan, Taiwan, USA, South Korea, Singapoure, India /// on drop les autres

styl = list(ram.Style)
count = list(ram.Country)
stars = list(ram.Stars)
c=0

for i in range(len(styl)) :
    i = i - c
    if styl[i] == 'Cup' :
        styl[i] = 1
    elif styl[i] == 'Pack' :
        styl[i] = 2
    elif styl[i] == 'Tray' :
        styl[i] = 3
    elif styl[i] == 'Bowl' :
        styl[i] = 4
    else :
        del(styl[i])
        del(count[i])
        del(stars[i])
        c +=1
        
    if count[i] == 'Japan' :
        count[i] = 1
    elif count[i] == 'Taiwan' :
        count[i] = 2
    elif count[i] == 'USA' :
        count[i] = 3
    elif count[i] == 'South Korea' :
        count[i] = 4
    elif count[i] == 'Singapoure' :
        count[i] = 5
    elif count[i] == 'India' :
        count[i] = 6
    else :
        del(styl[i])
        del(count[i])
        del(stars[i])
        c+=1
    
    if stars[i] == 'Unrated' :
        del(styl[i])
        del(count[i])
        del(stars[i])
        c += 1 

In [ ]:
Style = pd.DataFrame(styl)
Country = pd.DataFrame(count)
Stars = pd.DataFrame(stars)

ram = pd.DataFrame({'Style': styl, 'Country': count, 'Stars': stars})

In [ ]:
ram_train = ram.sample(frac=0.8, random_state=1) # 80% des données avec frac=0.8
ram_test = ram.drop(ram_train.index)

In [ ]:
donnee = []

for i in range(len(styl)) :
    donnee.append([styl[i], count[i]])
    #On créer les données d'entraînements
x_train = donnee    
y_train = stars

x_test = ram_test.drop(['Stars'], axis=1) #Puis celles de test
y_test = ram_test.drop(['Style','Country'], axis=1)

# Lancement du réseau de neurones sur le dataset des ramens

x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
for i in range(len(x_train)) :
    for j in range(0,2) : 
         x_train[i][j] = float(x_train[i][j])

In [ ]:
for i in range(len(y_train)) : 
    y_train[i] = float(y_train[i])

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 1, 2)

In [ ]:
net = Network()
net.add(FCLayer(2, 30))               
net.add(ActivationLayer(ReLU, dReLU))
net.add(FCLayer(30, 10))                   
net.add(ActivationLayer(ReLU, dReLU))


net.use(mse, mse_prime)
net.fit(x_train, y_train, epochs=20, learning_rate=0.1)